In [42]:
# Preamble.

# Make sure to pip install all these packages and run under a virtual env: https://janakiev.com/blog/jupyter-virtual-envs/
import math
import pint

unit = pint.UnitRegistry()

# Constants we want to use.
earth_acceleration = 9.81 * unit('meter / second**2')

# Dry air at 20C; see https://en.wikipedia.org/wiki/Density_of_air.
rho_air = 1.2041 * unit('kilogram / meter**3')

# Density of Aluminium; see https://en.wikipedia.org/wiki/Aluminium
rho_alu = 2.7 * unit('g / cm**3')


In [43]:
# Cylinder dimensions and configuration.

# Length.
cylinder_length = 16 * unit.kilometer

# Radius.
cylinder_radius = 3 * unit.kilometer

# Rotation speed in rotations / hour.
cylinder_rotation = 30.0 / unit.hour

# This is the wall thickness.
shell_thickness = 50 * unit.meter

# If we wanted to move it, how strong are our engines? For reference, the total Space Shuttle Engines plus SRBs provided ~30 MN
engine_thrust = 30000 * unit('meganewton')

# We tile our inner surface evenly into square subdivisions with this length.
subdivision_sidelength = 2 * unit.kilometer

In [45]:
# Derived geometrical and physical stats.

# Does not include the end caps.
def cylinder_surface_area(r: unit, h: unit) -> unit:
    return 2.0 * math.pi * r * h

def solid_cylinder_volume(r: unit, h: unit) -> unit:
    return r * r * math.pi * h

def hollow_cylinder_volume(inner_r: unit, outer_r: unit, h: unit) -> unit:
    return (outer_r*outer_r - inner_r*inner_r) * math.pi * h

# The 'equatorial' circle length.
cylinder_circumference = 2.0 * math.pi * cylinder_radius

cylinder_inner_area = cylinder_surface_area(cylinder_radius, cylinder_length)
cylinder_outer_area = cylinder_surface_area(cylinder_radius + shell_thickness, cylinder_length)
cylinder_volume = solid_cylinder_volume(cylinder_radius, cylinder_length)

print(f'Cylinder circumference:   {cylinder_circumference:.2f}')
print(f'Cylinder inner area:      {cylinder_inner_area:.2f}')
print(f'Cylinder outer area:      {cylinder_outer_area:.2f}')
print(f'Cylinder volume:          {cylinder_volume:.2f}')


surface_speed = cylinder_circumference * cylinder_rotation
print(f'Surface speed:            {surface_speed:.2f} / {surface_speed.to('meter / second'):.2f}')

centripetal_acceleration = (surface_speed**2 / cylinder_radius).to('meter / second**2')
relative_gs = centripetal_acceleration  / earth_acceleration

print(f'Centripetal acceleration: {centripetal_acceleration:.5f} / {relative_gs:.5f} Gs')

atmospheric_weight = (cylinder_volume * rho_air).to('megaton')
print(f'Atmospheric weight:       {atmospheric_weight:.2f}')

shell_volume = hollow_cylinder_volume(cylinder_radius, cylinder_radius+shell_thickness, cylinder_length)
print(f'Shell volume:             {shell_volume:.2f}')

shell_weight = (shell_volume * rho_alu).to('megaton')
print(f'Shell weight:             {shell_weight:.2f}')

dry_weight = shell_weight + atmospheric_weight
print(f'Dry weight:               {dry_weight:.2f}')

engine_acceleration = (engine_thrust / dry_weight).to('meter / second**2')
print(f'Engine acceleration:      {engine_acceleration:.10f} / {engine_acceleration / earth_acceleration:.10f} Gs')

Cylinder circumference:   18.85 kilometer
Cylinder inner area:      301.59 kilometer ** 2
Cylinder outer area:      306.62 kilometer ** 2
Cylinder volume:          452.39 kilometer ** 3
Surface speed:            565.49 kilometer / hour / 157.08 meter / second
Centripetal acceleration: 8.22467 meter / second ** 2 / 0.83840 dimensionless Gs
Atmospheric weight:       600.45 megaton
Shell volume:             15.21 kilometer ** 3
Shell weight:             45254.66 megaton
Dry weight:               45855.11 megaton
Engine acceleration:      0.0007211702 meter / second ** 2 / 0.0000735138 dimensionless Gs


In [64]:
# Calculation of potential living space.

# Let's tile evenly.
subdivisions_longitude = math.floor(cylinder_length / subdivision_sidelength)
subdivisions_latitude = math.floor(cylinder_circumference / subdivision_sidelength)
subdivisions_total = subdivisions_latitude*subdivisions_longitude

subdivision_area = subdivision_sidelength*subdivision_sidelength
subdivision_total_area = subdivision_area * subdivisions_total

# Each tile is split into four parts: residential, recreation, reservation / park and agricultural. The reservation
# park also includes water storage.
# Based on the work by MGeog2022; see
# https://upload.wikimedia.org/wikipedia/commons/f/f5/O_Neill_cylinder-Island_Three-Example_for_nearly_77_million_population_in_each_cylinder.png
# In this work, the space was evenly split, however, we can split it any way we see fit.
subdivision_ratios = {
    'residential': 0.2,
    'recreation': 0.1,
    'reservation': 0.4,
    'agriculture': 0.3
}
assert(sum(subdivision_ratios.values()) <= 1)

subdivision_residential_area = subdivision_total_area * subdivision_ratios['residential']
subdivision_recreation_area = subdivision_total_area * subdivision_ratios['recreation']
subdivision_reservation_area = subdivision_total_area * subdivision_ratios['reservation']
subdivision_agricultural_area = subdivision_total_area * subdivision_ratios['agriculture']

print(f'Subdivisions:             {subdivisions_longitude}x{subdivisions_latitude}={subdivisions_total}')
print(f'Total livable space:      {subdivisions_total*subdivision_area} ({subdivisions_total})')
print(f'Residential area:         {subdivision_residential_area:.1f} ({math.ceil(subdivision_residential_area / subdivision_area)})')
print(f'Recreation area:          {subdivision_recreation_area:.1f} ({math.ceil(subdivision_recreation_area / subdivision_area)})')
print(f'Reservation area:         {subdivision_reservation_area:.1f} ({math.ceil(subdivision_reservation_area / subdivision_area)})')
print(f'Agriculture area:         {subdivision_agricultural_area:.1f} ({math.ceil(subdivision_agricultural_area / subdivision_area)})')

Subdivisions:             8x9=72
Total livable space:      288 kilometer ** 2 (72)
Residential area:         57.6 kilometer ** 2 (15)
Recreation area:          28.8 kilometer ** 2 (8)
Reservation area:         115.2 kilometer ** 2 (29)
Agriculture area:         86.4 kilometer ** 2 (22)
